In [12]:
import os
import torch as T
import torchvision as TV
import torchaudio as TA
import cv2
import numpy as np
import random
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch import optim
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp
from glob import glob
import albumentations as A
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix
from pathlib import Path
import segmentation_models_pytorch as smp

In [13]:
T.manual_seed(22)
np.random.seed(22)
random.seed(22)

In [14]:
# ---------------------- DEVICE -----------------------
device = T.device("cuda" if T.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [15]:
# ---------------------- Paths -----------------------
train_images = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\train_new"
train_masks = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\trainlabel_new"
validation_images = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\validation_new"
validation_masks = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\validationlabel_new"
test_images = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\test_new"
test_masks = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\testlabel_new"

In [16]:
# ---------------------- Simple Transform -----------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ---------------------- Simplified Dataset Class -----------------------
from glob import glob
import torch
from torch.utils.data import Dataset, DataLoader
import os
import cv2
import numpy as np
from tqdm import tqdm

class SimpleSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, dataset_type="Unknown"):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.dataset_type = dataset_type
        self.image_files = sorted(glob(os.path.join(image_dir, "*.jpg")))
        self.mask_files = sorted(glob(os.path.join(mask_dir, "*.png")))
        self._verify_file_pairs()
        
    def _verify_file_pairs(self):
        if len(self.image_files) != len(self.mask_files):
            raise ValueError(f"Mismatched counts in {self.dataset_type} dataset: {len(self.image_files)} images vs {len(self.mask_files)} masks")
            
        for img_path, mask_path in tqdm(zip(self.image_files, self.mask_files), total=len(self.image_files), desc=f"Verifying {self.dataset_type} File Pairs 🔍"):
            img_name = os.path.splitext(os.path.basename(img_path))[0]
            mask_name = os.path.splitext(os.path.basename(mask_path))[0]
            if img_name != mask_name:
                raise ValueError(f"Filename mismatch in {self.dataset_type} dataset: {img_name} vs {mask_name}")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img = cv2.cvtColor(cv2.imread(self.image_files[idx]), cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_files[idx], cv2.IMREAD_GRAYSCALE)
        mask = (mask > 127).astype(np.uint8)
        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)
        
        mask = T.from_numpy(mask).long()
        
        return img, mask, self.image_files[idx]

def custom_collate_fn(batch):
    images = torch.stack([item[0] for item in batch])
    masks = torch.stack([item[1] for item in batch])
    filenames = [item[2] for item in batch]
    return images, masks, filenames

# ---------------------- DataLoaders -----------------------
train_dataset = SimpleSegmentationDataset(
    image_dir=train_images,
    mask_dir=train_masks,
    transform=transform,
    dataset_type="Training"
)
val_dataset = SimpleSegmentationDataset(
    image_dir=validation_images,
    mask_dir=validation_masks,
    transform=transform,
    dataset_type="Validation"
)
test_dataset = SimpleSegmentationDataset(
    image_dir=test_images,
    mask_dir=test_masks,
    transform=transform,
    dataset_type="Testing"
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    collate_fn=custom_collate_fn
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=custom_collate_fn
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=custom_collate_fn
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

Verifying Testing File Pairs 🔍: 100%|██████████████████████████████████████████| 1200/1200 [00:00<00:00, 54541.13it/s]

Training samples: 1600
Validation samples: 352
Testing samples: 1200


In [ ]:
# ---------------------- Model -----------------------
CUSTOM_SAVE_ROOT = Path(r"D:\AAU Internship\Code\UNet-Models")
os.makedirs(CUSTOM_SAVE_ROOT, exist_ok=True)

model = smp.Unet(
    encoder="efficientnet-b5",
    encoder_weights="imagenet",
    encoder_depth=4,
    decoder_use_batchnorm='inplace',
    decoder_attention_type='scse',
    decoder_channels=[256, 128, 64, 32],
    in_channels=3,
    classes=2,
    activation=None,
    center=True,
).to(device)

# ---------------------- Loss Function -----------------------
class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.7, beta=0.3, gamma=0.75, smooth=1e-6):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.smooth = smooth

    def update_hyperparams_by_epoch(self, epoch):
        steps = epoch // 5
        self.alpha = max(0.4, 0.7 - 0.03*steps)
        self.beta = 1 - self.alpha
        self.gamma = min(1.5, 0.5 + 0.1*steps)

    def forward(self, preds, targets):
        targets_one_hot = F.one_hot(targets, num_classes=preds.shape[1]).permute(0, 3, 1, 2).float()
        probs = F.softmax(preds, dim=1)
        dims = (0, 2, 3)
    
        TP = T.sum(probs * targets_one_hot, dims)
        FP = T.sum(probs * (1 - targets_one_hot), dims)
        FN = T.sum((1 - probs) * targets_one_hot, dims)
    
        Tversky = (TP + self.smooth) / (TP + self.alpha * FP + self.beta * FN + self.smooth)
        return T.mean((1 - Tversky) ** self.gamma)

loss_fn = FocalTverskyLoss().to(device)

# ---------------------- Metrics -----------------------
def compute_metrics(preds, targets):
    with T.no_grad():
        pred_labels = T.argmax(preds, dim=1).cpu().numpy().flatten()
        targets = targets.cpu().numpy().flatten()
        ious = []
        for cls in [0, 1]:
            intersection = ((pred_labels == cls) & (targets == cls)).sum()
            union = ((pred_labels == cls) | (targets == cls)).sum()
            ious.append(intersection / (union + 1e-6))
        class_acc = []
        for cls in [0, 1]:
            mask = (targets == cls)
            if mask.sum() > 0:
                class_acc.append((pred_labels[mask] == cls).mean())
        mPA = np.mean(class_acc) * 100
        cm = confusion_matrix(targets, pred_labels)
        TN, FP, FN, TP = cm.ravel()
        return {
            "Accuracy": 100 * accuracy_score(targets, pred_labels),
            "mPA": mPA,
            "Crop IoU": 100 * ious[1],
            "mIoU": 100 * np.mean(ious),
            "Precision": 100 * precision_score(targets, pred_labels, zero_division=0),
            "Recall": 100 * recall_score(targets, pred_labels, zero_division=0),
            "F1-Score": 100 * f1_score(targets, pred_labels, zero_division=0),
            "FNR": 100 * (FN / (FN + TP + 1e-6))
        }

# ---------------------- Training Setup -----------------------
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
MODEL_PATH = CUSTOM_SAVE_ROOT / "best_mPA_model.pth"
best_mPA = -1  # Initialize to negative value for maximization
PRIMARY_METRIC = "mPA"  # Primary metric for model selection

# ---------------------- Training & Validation -----------------------
def TrainUNet(model, dataloader, loss_fn, optimizer, epoch):
    model.train()
    running_loss = 0
    all_preds, all_targets = [], []
    loss_fn.update_hyperparams_by_epoch(epoch)
    loop = tqdm(dataloader, desc=f"Epoch {epoch} [Train]")

    for batch in loop:
        inputs, targets, _ = batch  # Unpack tuple: images, masks, filenames (ignored)
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        all_preds.append(outputs.detach().cpu())
        all_targets.append(targets.detach().cpu())
        loop.set_postfix(loss=loss.item())

    avg_loss = running_loss / len(dataloader)
    metrics = compute_metrics(T.cat(all_preds), T.cat(all_targets))

    T.cuda.empty_cache()
    
    return avg_loss, metrics

def ValidateUNet(model, dataloader, loss_fn):
    model.eval()
    running_loss = 0
    all_preds, all_targets = [], []
    loop = tqdm(dataloader, desc="Validating")

    with T.no_grad():
        for batch in loop:
            inputs, targets, _ = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            running_loss += loss.item()
            all_preds.append(outputs.detach().cpu())
            all_targets.append(targets.detach().cpu())
            loop.set_postfix(loss=loss.item())

    avg_loss = running_loss / len(dataloader)
    metrics = compute_metrics(T.cat(all_preds), T.cat(all_targets))
    
    T.cuda.empty_cache()

    return avg_loss, metrics

# ---------------------- Main Training -----------------------
num_epochs = 50
for epoch in range(1, num_epochs + 1):
    train_loss, train_metrics = TrainUNet(model, train_dataloader, loss_fn, optimizer, epoch)
    val_loss, val_metrics = ValidateUNet(model, val_dataloader, loss_fn)

    if val_metrics[PRIMARY_METRIC] > best_mPA:
        best_mPA = val_metrics[PRIMARY_METRIC]
        T.save(model.state_dict(), str(MODEL_PATH))
        print(f"✅ New best {PRIMARY_METRIC}: {best_mPA:.2f}% | Saved to: {MODEL_PATH}")

    print(f"\n📊 Epoch {epoch} Summary:")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    for k, v in val_metrics.items():
        print(f"{k}: {v:.2f}%")

    T.cuda.empty_cache()

# ---------------------- Final Report -----------------------
print(f"\n🎯 === Best Model Summary ===")
print(f"Best {PRIMARY_METRIC}: {best_mPA:.2f}% → {MODEL_PATH}")

# ---------------------- Testing -----------------------
print("\n🧪 === Testing Best Model ===")
model.load_state_dict(T.load(str(MODEL_PATH)))
test_loss, test_metrics = ValidateUNet(model, test_dataloader, loss_fn)
print(f"\n📌 Best {PRIMARY_METRIC} Model Test Results:")
for k, v in test_metrics.items():
    print(f"{k}: {v:.2f}%")

Validating: 100%|██████████████████████████████████████████████████████████| 88/88 [00:21<00:00,  4.17it/s, loss=0.164]


✅ New best mPA: 95.33% | Saved to: D:\AAU Internship\Code\UNet-Models\best_mPA_model.pth

📊 Epoch 1 Summary:
Train Loss: 0.3093 | Val Loss: 0.1690
Accuracy: 98.90%
mPA: 95.33%
Crop IoU: 89.61%
mIoU: 94.20%
Precision: 98.55%
Recall: 90.81%
F1-Score: 94.52%
FNR: 9.19%


Validating: 100%|██████████████████████████████████████████████████████████| 88/88 [00:20<00:00,  4.38it/s, loss=0.122]


✅ New best mPA: 97.75% | Saved to: D:\AAU Internship\Code\UNet-Models\best_mPA_model.pth

📊 Epoch 2 Summary:
Train Loss: 0.1446 | Val Loss: 0.1247
Accuracy: 99.30%
mPA: 97.75%
Crop IoU: 93.45%
mIoU: 96.33%
Precision: 97.44%
Recall: 95.80%
F1-Score: 96.61%
FNR: 4.20%


Validating: 100%|██████████████████████████████████████████████████████████| 88/88 [00:20<00:00,  4.21it/s, loss=0.113]


...

Validating: 100%|██████████████████████████████████████████████████████████| 88/88 [00:19<00:00,  4.50it/s, loss=0.112]


✅ New best mPA: 98.66% | Saved to: D:\AAU Internship\Code\UNet-Models\best_mPA_model.pth

📊 Epoch 4 Summary:
Train Loss: 0.1132 | Val Loss: 0.1143
Accuracy: 99.43%
mPA: 98.66%
Crop IoU: 94.68%
mIoU: 97.02%
Precision: 96.86%
Recall: 97.68%
F1-Score: 97.27%
FNR: 2.32%


Validating: 100%|█████████████████████████████████████████████████████████| 88/88 [00:19<00:00,  4.50it/s, loss=0.0807]


✅ New best mPA: 99.08% | Saved to: D:\AAU Internship\Code\UNet-Models\best_mPA_model.pth

📊 Epoch 5 Summary:
Train Loss: 0.0705 | Val Loss: 0.0819
Accuracy: 99.39%
mPA: 99.08%
Crop IoU: 94.40%
mIoU: 96.86%
Precision: 95.61%
Recall: 98.68%
F1-Score: 97.12%
FNR: 1.32%


Epoch 6 [Train]:  32%|████████████████▎                                  | 128/400 [01:23<03:22,  1.34it/s, loss=0.058]

In [19]:
# ---------------------- Mask Generation -----------------------
MASK_FOLDER_NAME = "Crop_Masks"
MASK_OUTPUT_DIR = Path.cwd() / MASK_FOLDER_NAME
for split in ['train', 'val', 'test']:
    (MASK_OUTPUT_DIR / split).mkdir(parents=True, exist_ok=True)

def save_segmentation_masks(model, train_dataloader, val_dataloader, test_dataloader, model_path, output_dir, device):
    model.load_state_dict(T.load(str(model_path)))
    model.eval()
    dataloader_splits = [
        (train_dataloader, 'train'),
        (val_dataloader, 'val'),
        (test_dataloader, 'test')
    ]
    mask_counter = 0
    
    with T.no_grad():
        for dataloader, split in tqdm(dataloader_splits, desc="Processing datasets", leave=False):
            split_dir = output_dir / split
            for batch in tqdm(dataloader, desc=f"Processing {split} batch", leave=False):
                inputs, _, filenames = batch
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                outputs = model(inputs)
                pred_labels = T.argmax(outputs, dim=1).cpu().numpy()
                for i in range(batch_size):
                    mask = pred_labels[i]
                    mask = (mask * 255).astype(np.uint8)
                    image_name = Path(filenames[i]).stem + ".png"  # Save as .png
                    mask_path = str(split_dir / image_name)
                    cv2.imwrite(mask_path, mask)  # cv2 automatically writes as PNG with .png extension
                    mask_counter += 1
    
    print(f"\n🎉 Saved {mask_counter} segmentation masks to {output_dir}")

print("\n🖼️ Generating and saving segmentation masks...")
save_segmentation_masks(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    model_path=CUSTOM_SAVE_ROOT / "best_mPA_model.pth",
    output_dir=MASK_OUTPUT_DIR,
    device=device
)

T.cuda.empty_cache()



🖼️ Generating and saving segmentation masks...


Processing test batch: 100%|█████████████████████████████████████████████████████████| 300/300 [01:31<00:00,  3.29it/s]
                                                                                                                       


🎉 Saved 3152 segmentation masks to D:\AAU Internship\Code\Crop_Masks
